# Toronto neighbourhood clustering

## Import packages

In [48]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import geocoder 
from geopy.geocoders import Nominatim 

import pgeocode

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup #to parse data from webpages

## Extract data from wikipedia page

We want to start by extracting the existing table in this Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. First, we define the url in a variable, and we use BeautifulSoup to parse the web page code. Then, we find the table within the code.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(url).text

soup = BeautifulSoup(wiki_page, 'lxml')
wiki_table = soup.find('table',{'class':'wikitable'})

Now, we will iterate along the table in order to extract its elements and write them into a pandas dataframe. The first thing we will do is define the dataframe, extrating first the column names from the original table.

In [4]:
header = []

for col_name in wiki_table.findAll('tr')[0].findAll('th'):
    header.append(col_name.contents[0][0:-1])

df = pd.DataFrame(columns = header)

Now, we will iterate along each row and extract its information. Notice that there might be rows that we do not extract, because they correspond to postal codes without a Borough assigned. Thus, after all the processing, we reset the dataframe indices. 

In [5]:
for i in range(1,len(wiki_table.findAll('tr'))):
    row = wiki_table.findAll('tr')[i].findAll('td')
    row_list = []
    
    if row[1].contents[0][0:-1] != 'Not assigned':
        for j in range(len(row)):
            row_item = row[j]
            row_list.append(row_item.contents[0][0:-1])
        
    
        df.loc[i-1] = row_list
        
df.reset_index(inplace = True, drop = True)

To finish with this part, we replace the slashes in the 'Neighborhood' column by commas.

In [6]:
for i in range(len(df['Neighborhood'])):
    df['Neighborhood'][i] = df['Neighborhood'][i].replace(' /', ',')

Here are the 15 first rows of the resulting dataframe.

In [7]:
df.head(15)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


It has 103 rows (corresponding to the 103 registered postal codes in Toronto) and 3 columns

In [8]:
df.shape

(103, 3)

## Add coordinates of each postal code

We use pgeocode in order to search the coordinates of each postal code, and add them to the dataframe. 

In [66]:
geolocator = pgeocode.Nominatim('ca')

for i in range(df.shape[0]):
    pc = df.loc[i,'Postal code']
    query = geolocator.query_postal_code(pc)
    lat = query.latitude
    long = query.longitude
    
    df.loc[i,'Latitude'] = lat
    df.loc[i, 'Longitude'] = long
    

In [67]:
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
7,M3B,North York,Don Mills,43.7450,-79.3590
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


A fast examination of the dataset reveals that the only postal code that has no coordinates assigned is the M7R postal code, corresponding to the Canada Post Gateway Processing Centre. Its latitude and longitud are 43.6364982, -79.6176319.

In [75]:
df.loc[76,'Latitude'] = 43.6364
df.loc[76,'Longitude'] = -79.6176

In [76]:
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
7,M3B,North York,Don Mills,43.7450,-79.3590
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


## Cluster Neighborhoods

We are going to perform cluster analysis based on venue information of each postal code obtained from the Foursquare API. Let us first visualize all the Neighborhoods on a map, based on the coordinates we have added before.

In [160]:
latitude, longitude = 43.6510, -79.3470#latitude and longitude of the city of Toronto 

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [128]:
CLIENT_ID = 'LXSJT23M5YNOJJEUE4OZI0VU45VY2M1BFRCRRCGRFKW2XACM' # your Foursquare ID
CLIENT_SECRET = '3NECBGQYATTOMBZSWU43DXUSEE4OXBFNT3YQA4TCZ5RIGZKN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LXSJT23M5YNOJJEUE4OZI0VU45VY2M1BFRCRRCGRFKW2XACM
CLIENT_SECRET:3NECBGQYATTOMBZSWU43DXUSEE4OXBFNT3YQA4TCZ5RIGZKN


#### Define function to get venues of each postal code

In [129]:
def getNearbyVenues(boroughs, neighbourhoods, latitudes, longitudes, radius=500, LIMIT = 30):
    
    venues_list=[]
    for bor, neigh, lat, lng in zip(boroughs, neighbourhoods, latitudes, longitudes):
        print(neigh)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            bor,
            neigh,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough','Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [130]:
venues_df = getNearbyVenues(df['Borough'], df['Neighborhood'], df['Latitude'], df['Longitude'], radius=500, LIMIT = 30)

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [153]:
venues_df.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,North York,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,North York,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,North York,Victoria Village,43.7276,-79.3148,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,North York,Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [154]:
venues_df.shape

(1331, 8)

In [155]:
venues_df.groupby('Neighborhood').count()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Agincourt,5,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",6,6,6,6,6,6,6
Bayview Village,4,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",28,28,28,28,28,28,28
Berczy Park,30,30,30,30,30,30,30
"Birch Cliff, Cliffside West",4,4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",30,30,30,30,30,30,30
Business reply mail Processing CentrE,14,14,14,14,14,14,14


In [156]:
print('There are {} uniques categories.'.format(len(venues_df['Venue Category'].unique())))

There are 220 uniques categories.


### Analyze each neighbourhood

In [157]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Yoga Studio,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [136]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [137]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1  Latin American Restaurant   0.2
2               Skating Rink   0.2
3        Shanghai Restaurant   0.2
4            Badminton Court   0.2


----Alderwood, Long Branch----
            venue  freq
0             Gym  0.12
1  Sandwich Place  0.12
2        Pharmacy  0.12
3     Pizza Place  0.12
4     Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0  Middle Eastern Restaurant  0.17
1                Pizza Place  0.17
2                Coffee Shop  0.17
3   Mediterranean Restaurant  0.17
4              Deli / Bodega  0.17


----Bayview Village----
         venue  freq
0         Park  0.25
1  Flower Shop  0.25
2  Gas Station  0.25
3        Trail  0.25
4  Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.11
1     Thai Restaurant  0.07
2         Coffee Shop  0.07
3          Restaura

           venue  freq
0       Pharmacy  0.33
1           Park  0.17
2  Grocery Store  0.17
3   Skating Rink  0.17
4           Bank  0.17


----Kennedy Park, Ionview, East Birchmount Park----
                venue  freq
0         Coffee Shop  0.25
1  Light Rail Station  0.08
2    Department Store  0.08
3            Bus Line  0.08
4      Sandwich Place  0.08


----Kensington Market, Chinatown, Grange Park----
                   venue  freq
0                   Café  0.10
1  Vietnamese Restaurant  0.10
2     Chinese Restaurant  0.07
3     Mexican Restaurant  0.07
4          Grocery Store  0.03


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
               venue  freq
0        Pizza Place  0.08
1  Mobile Phone Shop  0.08
2      Shopping Mall  0.08
3        Supermarket  0.08
4     Sandwich Place  0.08


----Lawrence Manor, Lawrence Heights----
              venue  freq
0    Clothing Store  0.33
1    Cosmetics Shop  0.07
2  Toy / Game Store  0.07
3          

We are going to create a dataframe including, for each neighborhood, the top 3 most common venues. We choose 3 because we see that, for some neighbourhoods, there are only 2 or 3 types of venues.

In [138]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [139]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Skating Rink,Shanghai Restaurant,Latin American Restaurant
1,"Alderwood, Long Branch",Gym,Pharmacy,Sandwich Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Fried Chicken Joint,Coffee Shop,Pizza Place
3,Bayview Village,Park,Flower Shop,Gas Station
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Thai Restaurant


In [140]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 1, 3, 3, 3, 3, 3, 3], dtype=int32)

In [147]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [148]:
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,0.0,Park,Food & Drink Shop,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.7276,-79.3148,1.0,Pizza Place,Intersection,Park
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,3.0,Coffee Shop,Breakfast Spot,Yoga Studio
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,3.0,Clothing Store,Cosmetics Shop,Toy / Game Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,3.0,Gym,Beer Bar,Boutique
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282,1.0,Pharmacy,Park,Bank
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930,3.0,Construction & Landscaping,Home Service,Women's Store
7,M3B,North York,Don Mills,43.7450,-79.3590,1.0,Gym,Park,Construction & Landscaping
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094,3.0,Pizza Place,Fast Food Restaurant,Intersection
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,3.0,Coffee Shop,Café,Theater


We notice that there are some neighbours without venues nor category. We discard them from the dataframe.

In [158]:
toronto_merged.dropna(inplace = True)

In [166]:
latitude, longitude = 43.6510, -79.3470#latitude and longitude of the city of Toronto 

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [167]:
for lat, lng, borough, neighborhood in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighborhood']):

#for lat, lng, borough, neighborhood, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = '{} - {}'.format(neighborhood, borough)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       

map_clusters

In [176]:
latitude, longitude = 43.6510, -79.3470#latitude and longitude of the city of Toronto 

# create map of New York using latitude and longitude values
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)  
    
map_clusters

## Analyze each cluster

Let us find out what the clusters consist on.

In [183]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Parkwoods,0.0,Park,Food & Drink Shop,Fast Food Restaurant
27,Hillcrest Village,0.0,Park,Residential Building (Apartment / Condo),Department Store
61,Lawrence Park,0.0,Park,Photography Studio,Deli / Bodega
64,Weston,0.0,Park,Department Store,Falafel Restaurant
69,"High Park, The Junction South",0.0,Park,Residential Building (Apartment / Condo),Department Store


In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Victoria Village,1.0,Pizza Place,Intersection,Park
5,Islington Avenue,1.0,Pharmacy,Park,Bank
7,Don Mills,1.0,Gym,Park,Construction & Landscaping
13,Don Mills,1.0,Gym,Park,Construction & Landscaping
16,Humewood-Cedarvale,1.0,Field,Trail,Hockey Arena
21,Caledonia-Fairbanks,1.0,Park,Women's Store,Sporting Goods Shop
22,Woburn,1.0,Korean Restaurant,Women's Store,Dessert Shop
31,"Dufferin, Dovercourt Village",1.0,Grocery Store,Bakery,Park
35,East Toronto,1.0,Park,Coffee Shop,Convenience Store
36,"Harbourfront East, Union Station, Toronto Islands",1.0,Park,Harbor / Marina,Music Venue


In [185]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
32,Scarborough Village,2.0,Grocery Store,Department Store,Falafel Restaurant


In [186]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,"Regent Park, Harbourfront",3.0,Coffee Shop,Breakfast Spot,Yoga Studio
3,"Lawrence Manor, Lawrence Heights",3.0,Clothing Store,Cosmetics Shop,Toy / Game Store
4,"Queen's Park, Ontario Provincial Government",3.0,Gym,Beer Bar,Boutique
6,"Malvern, Rouge",3.0,Construction & Landscaping,Home Service,Women's Store
8,"Parkview Hill, Woodbine Gardens",3.0,Pizza Place,Fast Food Restaurant,Intersection
9,"Garden District, Ryerson",3.0,Coffee Shop,Café,Theater
10,Glencairn,3.0,Pizza Place,Fast Food Restaurant,Japanese Restaurant
11,"West Deane Park, Princess Gardens, Martin Grov...",3.0,Pizza Place,Tea Room,Construction & Landscaping
12,"Rouge Hill, Port Union, Highland Creek",3.0,Golf Course,History Museum,Women's Store
14,Woodbine Heights,3.0,Diner,Spa,Convenience Store


By taking a quick look at these lists, it might be convinient to categorize each cluster as:
- Cluster 1: parks
- Cluster 2: gyms and sport
- Cluster 4: restaurants, coffee shops and leisure 